# Notes on Manabe/Wetherald 1967
## Thermal Equilibrium of the Atmosphere with a Given Distribution of Relative Humidity


## 1. Introduction

- It's a follow up study
- Preceded by papers:
   - [1964 Manabe/Strickler](https://www.gfdl.noaa.gov/bibliography/related_files/sm6401.pdf): thermal equilibrium of the atmosphere with convective adjustment
      - Humidity correction factor was a given vertical distribution of absolute humidity, not relative, not as a function of temperature
   - [1961 Manabe & Möller](https://journals.ametsoc.org/view/journals/mwre/89/12/1520-0493_1961_089_0503_otreah_2_0_co_2.xml): radiative convective equilibrium
      - mentions this [1956 paper by Phillips](https://empslocal.ex.ac.uk/people/staff/gv219/classics.d/Phillips56.pdf) modeling energy flows numerically assuming a heat source near the equator and heat sink near the poles.
   - [1965 Manabe et al](https://journals.ametsoc.org/view/journals/mwre/93/12/1520-0493_1965_093_0769_scoagc_2_3_co_2.xml): Use computation (numerical integration) to model radiative transfer in the atmosphere including the hydrologic cycle
      - Used a climatological distribution of absolute humidity as approximation
    
- Why change the model from absolute to RH?
  - Absolute humidity is highly coupled to temperature
  - Average RH latitude-height distributions appear similar in winter and summer, whereas AH equivalents do not
    - They don't provide a plot for the latter
  - Maybe you can assume a given (fixed) vertical distribution of RH since it doesn't change much seasonally
  - Removing an unknown saves effort for parameters that are dynamic & saves computation cycles
  - If RH distribution is static, absolute humidity is a simple function of temperature (for a fixed altitude+latitude)
  - If atmospheric temperatures are higher, then the absolute humidity at any given altitude is higher
  - Long-wave infrared (LWIR) emissions to space mostly happen at the altitude surface where there isn't much more LWIR-absorbent substance (e.g. humidity) at a higher altitude to absorb outgoing photons and re-emit them downward
    - Higher absolute humidity at all altitudes -> effective altitude of outbound LWIR emission gets higher
      - Higher altitude => larger surface area of LWIR emission, A = 4 * pi * r^2
      - _If the surface area grows but the outgoing LWIR power is fixed, power density (W/m2) decreases_
    - What are the effects on the convective equilibrium if this occurs?
      
- Next step (this paper): numerically solve the fully coupled radiative transfer and hydrological cycle elements.
- But before doing so, compute a few radiative convective equilibria with fixed relative humidity to answer:
  1) How fast does the atmosphere equilibriate when RH is realistic but time-invariant?
  2) How do optical/radiative variables affect the atmosphere's equilibrium temperature under realistic but static RH conditions?
  3) From the atmosphere temperature calculated in 1) and 2), what's the surface equilibrium temp?
- Use those answers to sanity check more complex models

- Möller 1963 discussed CO2 content vs. magnitude of LWIR at earth's surface relating to equilibrium temperature
  - Relative vs. absolute humidity confusion
  - Similar order-of-magnitude dependence of equilibrium temperature on CO2 content vs papers from the past 15 years
  - "We should probably re-evaluate"


### Quick terminology check
What is relative vs absolute humidity?
- Absolute aka specific humidity: grams H2O vapor per m^3, not including water droplets
- Relative humidity (RH): density of vapor (not suspended droplets) in the air as a percentage of the saturation vapor pressure, determined by Clausius-Clapeyron equation (or similar)
- Mixing ratio: number of moles of something vs. everything else
  - Mixing ratio of water vapor: number of units of water vapor per unit of the rest of the air (moles or grams)
  - (simple - even though it sounds like some kind of convection dynamics parameter)
- Lapse rate: change in temperature with altitude
- Tropopause: point where the lapse rate goes to zero (inversion layer)
  - Above it is the stratosphere
  - Below it is the troposphere

## 2. Radiative convective equilibrium

This is a long section, stretches across 5 pages.

Setting up the T&Cs of the model at equilibrium:

  - net incoming solar and outgoing LWIR energy are equal where the atmosphere ends 
  - No temperature discontinuities are allowed
  - Critical lapse rate: 6.5C/km, above it convection/eddies will crush it back to 6.5C/km, below it radiative equilibrium is satisfied
  - Heat capacity of Earth's surface = 0 (model is simplified, ignores oceans)
  - New for this paper: RH in the atmosphere is some given vertical distribution (maybe compiled from radiosonde data?)

Some notes
- Radiation balance defines when we reach equilibrium, otherwise temperature would change
- What is lapse rate? Change in atmospheric temperature with altitude
- number 2: discontinuities imply either free energy that should dissipate, not good for a numerical simulation, or a solid insulating barrier which doesn't exist


Numerical formulation is as an initial value problem that they let asymptote to some equilibrium (see Appendix 1)
- "Marching computation": Figure 2
  - compute mixing ratio of water vapor r(time), (why does the water vapor mixing ratio matter?)
    - `r = 0.622*h*e_s(T) / (p - h*e_s(T))`
  - compute mean emissivity and absorptivity,
  - compute temperature change due to blackbody radiation,
  - timestep: compute next temperature from last temperature + temperature change from BB radiation,
  - update convective effects (?),
  - loop
- e_s(time) is saturation vapor pressure of water vapor as a function of temperature, h = relative humidity
- I is the "indexing of the finite differences in the vertical direction"
  - Seems like vertical axis is modeled in discrete altitude blocks

In [2]:
# Plotting the mixing ratio as a function of e_s
# %pip install matplotlib mlx
import matplotlib.pyplot as plt
import mlx.core as mx
p = 1000 # assume some static pressure
e_s = mx.arange(100) + 1 # saturation pressure is some value between 1-101 (arb units)
h = (mx.arange(100)/100) # relative humidity is 0.0 to 1.0
r = mx.outer(0.622*e_s / (p - h*e_s), h)

#fig, ax = plt.subplots()
#plt.imshow(r)
#cb = plt.colorbar()
#plt.xlabel("h?")
#plt.ylabel("e_s?")
# r = 0.622*h*e_s(T) / (p - h*e_s(T))


## 2. Radiative convective equilibrium (continued)
- Simplification from absolute -> relative humidity means absolute humidity is essentially directly computable from temperature if the RH distribution is fixed and known.
- Heat capacity of moist air is needed to convert RH to AH
  - `C'_p = C_p[1 + (L/C_p)*pd_wrt_T( (0.622*he_s(T)) / (p - he_s(T)) )]`
  - L is latent heat of evaporation
  - C_p is specific heat of air under constant pressure
  - Term in brackets after `1 +` is due to 'change of latent energy of the air'
- Other computations are copied directly from Manabe/Strickler 1964
  - LWIR flux, solar radiation depletion (?), mean absorptivity and emissivity
- Atmospheric absorbers are considered to be distributed according to a standard table
  - CO2, water vapor, ozone, cloud
- Water vapor distribution is relatively well known
  - : RH = h_*(Q-0.02)/(1-0.02)
  - where h_* is surface RH, = 0.77
  - p_* is surface pressure
  - Q = p/p_* (pressure as fraction of surface pressure)
  - But there's a catch - if Q<0.02, you get a negative RH value
    - Stratosphere is very dry, mixing ratio is 3e-6 g/(g air)
    - Minimum value of mixing ratio (r_min) should be this value
- CO2 distribution is not complicated - 0.0456% by weight or 300 ppm by volume
  - You can tell this paper was 60 years ago: CO2 mixing ratio is now 420 ppm (by volume)
- Ozone distribution was collected using 'chemiluminescent ozonesondes' between Sept 1963-Aug 1964
  - Between 25e4 and 200e4 cm/km at NTP, peaking around 20km altitude
- Cloud altitudes, amounts and albedo are presented in a tabular format
- Albedo of Earth's surface assumed to be 0.102 (10.2% reflected)
 
- [Hergesell](https://de.wikipedia.org/wiki/Hugo_Hergesell)'s problem (1919)
  - [Hergesell 1919](https://empslocal.ex.ac.uk/people/staff/gv219/classics.d/Hergesell19.pdf) (found via [this page](https://empslocal.ex.ac.uk/people/staff/gv219/classics.d/climate.html))
    - This paper is 105 years old
    - Fun note: Hergesell [is known](https://econterms.net/aero/Hugo_Hergesell) to have taken a balloon with Count von Zeppelin 1km above Svalbard in 1910 to measure temperature+humidity+pressure 
  - Pure radiative equilibrium without convection assuming fixed RH distribution
  - Problem: if you assume grey body radiation, atmosphere is extremely cold (190K or -90C at 10km) and isothermal at radiative equilibrium
    - "Self-amplification effect of water vapor on equilbrium temperature of the atmosphere"
    - Water content of atmosphere decreases with decreasing temperature -> greenhouse effect decreases with temperature
    - In short: models seem to show runaway cooling, which can't be right
  - Figure 5 shows different models:
    - All three models are similar above 20km, but diverge below
    - Radiative equilibrium (fixed relative humidity): way too cold near surface
    - Radiative equilibrium (fixed absolute humidity): still way too cold, even colder at 10km altitude
    - Radiative-convective equilibrium (fixed relative humidity): smoother decrease in temperature with altitude up to 15km
  - Convective adjustment (?) gets models closer to real distributions of temperature with altitude
    - Equations (2) and (3) used for humidity distribution in this case
    - Troposphere temperature and tropopause altitude would be too low due to water vapor feedback runaway without convective adjustment
- Claim: rate of convergence towards equilibrium should be lower for fixed RH distribution than fixed AH distribution
  1. Outgoing radiation is less sensitive to atmospheric temperature for a fixed RH distribution than fixed AH distribution
     - Average altitude of LWIR emission goes up with higher temperature (higher water vapor mixing ratio -> higher altitude of least water vapor density) and down with lower temperature
     - LWIR emission sphere 'inflates' with higher temperatures (storing energy?)
  2. Vertical distribution of RH is constant -> AH depends only on atmosphere temperature and variation in AH involves variation in latent energy of air
     - -> Effective heat capacity of air with some RH is higher than heat capacity of dry air, so it buffers temperature changes (more thermal mass -> slower approach to equilibrium)
     - Figure 6 appears to be from a simulation, but uses axis centered on 0 deviation from equilibrium temperature
       - Models perturbed by +/- 15C from equlibrium
       - No apparent day/night cycle in these models, must be averaged out
     - Three example models (rising and falling to equilibrium):
       - Atmosphere 1: Vertical distribution of AH constant (fastest convergence)
       - Atmosphere 2: Vertical distribution of RH constant, but heat capacity of water and vapor ignored in air
       - Atmosphere 3: Vertical distrubution of RH constant and heat capacity of air varies with temperature (via AH) (slowest convergence)
       - Atmosphere 2 takes 1.5x longer than 1 to return to equilibrium (within what cutoff?), 3 takes even longer
         - +15C Atmosphere 3 takes even longer to reach equilibrium than -15C A3
           - Takeaway: "Probably don't start with a very warm initial condition" as it might not converge as expected
       - Radiation condensation relaxation much slower than radiation relaxation, probably major factor in seasonal variation of atmospheric temperature
         - (Thought it was radiative-convective equilibrium?)
         - (Does this refer to condensation of water with temperature?)

## 3. Solar constant and radiative convective equilibrium
- Pretty short section: less than a full page
- How much power is the Sun putting out, and shouldn't it affect the Earth's temperature?
  - _Solar constant: the average amount of solar energy that reaches the Earth's upper atmosphere per unit of time and area_
  - AKA total solar irradiance
  - About 1.2 kW/m2 facing the Sun
  - Only about 1 kW/m2 makes it to the surface on a sunny day, depending on location
  - Solar constant varies (as measured from orbit) [by ~0.1% per decade](https://ntrs.nasa.gov/api/citations/19910003143/downloads/19910003143.pdf)
- Knowing the relationship between the two is useful: how do noise floor variations (solar constant, CO2 concentration, humidity) affect atmospheric temperature?
- Manabe et al model the surface equilibrium temperature as a function of the solar constant: whether the relationship is linear or otherwise, and against the current RH vs the previous AH model.
  - Result: with the relative humidity model, limited to middle latitudes, the solar constant has twice the effect as with the AH model
  - But this effect reduces as the temperatures decrease, and increases rapidly as they increase over 300K: "self-amplification" caused by phase change of water vapor.
  - Fig 8 also shows temperature variation with solar constant is highest near the surface. 
  - TODO: what is the unit LY/MIN? M&W vary it between 1.2-2.4. kW/m^2?

- Changes to the solar constant induce changes to the overall vertical emission power in the long-wave IR at the top of the atmosphere, but not as much as would be expected based on Stefan-Boltzmann blackbody emission ($\sigma*t^4$).
  - As with other perturbations, authors attribute this to modification in the average altitude of emission. "The atmosphere expands" and causes a buffering effect

## 4. Equilibrium temperature and atmospheric absorbers
- Absorbers like H2O, CO2, O3, clouds: what is the scale of their effects?

- Relative humidity of the troposphere (sub-tropopause, sub-stratosphere: where we live) : variations in the $h_*$ value between 0.2, 0.6, 1.0. Figure 11.
  - Takeaways:
    - Higher tropo RH -> higher tropo equilibrium temperature
    - Outgoing radiation doesn't change much with tropo RH
- Table 2: net upward radiation $R_L$ and surface equilibrium temp $T_*^e$ varies with RH at the surface $h_*$
  - Temperature increases 5-10C with RH increase: seems substantial
  - R_L increases by 2-3%, which is pretty small

- Water vapor in the stratosphere:
    - Water vapor injected by supersonic transport flights was considered, even though the supersonic bit ended up not panning out (Concorde cancellation)
 
- Fig 12 examination:
    - Larger Rmin in stratosphere -> warmer troposphere
    - Larger Rmin in stratosphere -> colder stratosphere
    - The stratosphere's equilibrium temperature depends on the stratosphere's mixing ratio more than the troposphere's equilibrium temperature does
- Table 3
    - Variation in equilibrium temperature at surface with stratospheric water vapor mixing ratio
          - 3e-6 g/g -> 288.4K
          - 15e-6 g/g -> 290.4K
          - 75e-6 g/g -> 296K
      - Mixing ratio in the actual atmosphere is estimated around 3e-6 g/g per Mastenbrook 1963 and others
      - "Going from 3e-6 to 75e-6 g/g would only increase the temperature by about 2.0C" - hmmm, this doesn't appear to be correct given table 3 shows an increase of 7.6 degrees?
      - Manabe et al doubt such an increase could occur from supersonic aircraft water emissions alone.
- Manabe et al 1965 concluded that mostly horizontal eddies in the atmosphere rapidly pull moisture from high and middle-latitude stratosphere by freezing and precipitating it near the equatorial tropopause
    - NB: those results should be taken with a grain of salt due to some numerical issues relating to vertical advection of water vapor near the tropopause
- Figure 13: Kind of a confusing graph. Are they showing that increasing Rmin in the stratosphere... increases the mixing ratio in the stratosphere? Perhaps this just shows that they can set an initial condition for Rmin and it will stabilize.

- Carbon dioxide.
- Not in the paper, but one might wonder by ~500 ppm (0.04%) of the atmosphere being CO2 matters. CO2 is so resonant in infrared light that most IR lasers for cutting/welding (ranging from 100W to many kW) use it as a lasing medium. If you shifted the IR spectrum down, CO2 might look about as red as bromine.
    - Möller 1963 discussed CO2 vs surface temperature & RH
    - Solution found by computing magnitudes of downward LWIR emission (from the sky to ground) corresponding to different CO2 levels and finding surface temperatures needed to equlibriate radiation flux at those levels.
    - Möller found a self-amplification effect existed when increasing RH as temperature increases
        - This can change the surface temperature "almost arbitrarily". In short: the amplification could have huge gain with a small increase in RH
- M&W try to reproduce this study 
    - Vertical distribution of cloudiness and RH same as in Section 2B
    - Values from other papers also added
    - Figure 9: Net upward LWIR at Earth's surface vs. surface temperature
    - It has a shallow negative slope in the present study's data, decreasing less than 50% for temperatures changing from ~270K to ~310K
        - Think about what this means.

- M&W analysis:
    - Dependence of net radiation on temperature is small, increasing or decreasing depending on the method of computation
    - Some analysis suggests increasing CO2 would reduce Earth's surface temperature, which doesn't align with current expectations
    - "Arguments based on this data are weak" because they only account for Earth's surface temperature; atmosphere could be hotter or colder
    - Atmosphere could essentially take over as the main absorber & emitter of heat, so the surface would be cold but dark (almost a Venus, but without the boiling surface and high pressure)
    - A more complete picture can be made by considering convection and radiative convective equilibrium.
- Table 5 / Figure 16: Surface temperature change and vertical temperature distributions with changes in CO2 content
    - 300-150ppm: more of a drop with fixed RH than fixed AH
    - 300-600ppm: Almost double the increase with fixed RH than with fixed AH
    - All cases: 0.03 to 0.56 degree difference between average cloudiness and clear conditions, but clear sky results in more change. Cloudy conditions seem to buffer or shield the surface
- M&W used a crude method of estimating overlap between 15 micron band of CO2 with rotation band of water vapor, but don't believe it should affect general conclusions
- "Interesting to note:" Surface temperature variation with CO2 content obtained by Möller 1963 and in this paper "with fixed RH" agree "reasonably well" (per Table 6)
    - Slightly confusing: variation in T with fixed AH from Möller and this paper agree, but the variation in T with fixed RH from this paper is a different value
    - Writing issue, seems clear what they mean

- Ozone
- Thermal equilibrium states computed for different distributions of ozone (Fig 17)
    - Data source: Herring and Borden 1965, which used ozonesondes in April, in only the Northern hemisphere
    - Not a lot of compute or sensing available at this time, so they had to specifically pick a few points on the globe where data existed.
        - In 2024, you can just download maps of ozone concentration vs altitude vs. latitude and longitude
    - Distribution in this case means distribution across altitude for three different latitudes: equator at 0N, north at 40N, and polar at 80N
        - Where did they go to take these measurements, Svalbard?
- Features observed:
    - Equator has the least ozone, peaking around 27km altitude
    - Polar region has the most, peaking around 18km altitude
    - Ozone at higher altitudes tends to stay around the same density everywhere, increasing overall concentration just pumps more lower in altitude
- Noteworthy features:
    - More ozone -> warmer troposphere and lower stratosphere, colder upper stratosphere
    - Ozone distribution only significantly changes equilibrium temperature in stratosphere, not troposphere
    - Confusing: "Ozone distribution of 0N [] tends to make the tropopause height higher and the tropopause profile sharper than those of 80N []...
- Table 7: Surface equilibrium temperature doesn't change much (287.9-290.3) between different latitudes (0,40,80N) with differing total O3 amounts (0.260, 0.351, 0.435 cm/STP)
- More of a "we thought about it and didn't go further" section.

- Overall: Unclear what interactions the authors are modeling for ozone here. No spectral absorption discussed. It looks like they're just repeating the data from Herring and Borden and normalizing it against the total amount measured by London (1962)?

- Surface albedo: if we change the surface reflectivity, what happens?
    - My guess: they model albedo as the fraction of incoming light converted directly into heat vs. reflected back up into atmosphere. Not accounting for emissivity
    - Figure 19: Vertical distribution of radiative convective equilibrium temperature at altitude vs. different albedo
        - At surface:
            - 0.0 -> 297.2K
            - 0.2 -> 276.4K
            - 0.4 -> 253.2K
            - 0.6 -> 227.0K
        - Authors' approximation: for every 1% increase in albedo, surface temperature increases by 1 deg, initial condition 300K for alpha=0.0
    - Larger albedo -> colder atmosphere temperature
    - Surface albedo changes change temperature more near the surface and less at higher altitudes, negligible at 30km (9mb level). Makes sense on a conductivity / intuitive level
    - Sensitivity to albedo in M&W's RH model is almost twice as large as sensitivity to albedo in the same model when using absolute humidity instead

- Cloudiness
  - Table 9 and Figure 20
  - Generally, more clouds -> lower equilibrium surface temperature
    - But this decreases with cloud altitude doesn't hold for cirrus
      - Cirrus cloud thinning might originate in this train of thought
  - Equilibrium temperature of atmosphere with average cloudiness is about 20.7C colder than for a clear atmosphere
      - Significant impact from clouds
      - Significantly larger than same temperature equilibrium in model using fixed absolute humidity distribution
    - Low and middle clouds are the primary drivers of temperature decrease overall, depending on cloud "blackness"

- Q: How are the M&W modeling clouds? Pure white reflectors, some arbitrary albedo/emissivity, internal convection, downward absorption/reflection, suspended water content?
    - Table 9: what is full black, half black, low/middle/high?
    - What is the mean free path of an LWIR photon through clouds vs clear atmosphere vs. humid air?
    - M&W: "Section 2b describes cloud albedo and gaseous absorbers"
  - M.S. includes discussion on height & blackness of cirrus & impact
